In [2]:
import os
import shutil
import time
import datetime as dt
import csv
import json
import pandas as pd

from resources import ConfigS3
define = ConfigS3()

import config as CONFIG
import boto3
client_s3 = boto3.client('s3',
                    region_name=CONFIG.S3_REGION,
                    aws_access_key_id=CONFIG.S3_ACCESS_KEY,
                    aws_secret_access_key=CONFIG.S3_SECRET_ACCESS_KEY
                    )

PERIOD_LENGTH = 5  # minutes
SOURCE_DIR = os.path.join('.', 'tom-tom-data')
TRAIN_DIR = os.path.join('.', 'train_tom_tom_data_with_velocity')

if not os.path.exists(TRAIN_DIR):
    os.makedirs(TRAIN_DIR)

def velocity_to_los(velocity):
    if velocity < 15:
        return 'F'
    elif velocity < 20:
        return 'E'
    elif velocity < 25:
        return 'D'
    elif velocity < 30:
        return 'C'
    elif velocity < 35:
        return 'B'
    else:
        return 'A'


def los_to_velocity(los):
    los_to_velocity = {
        'A': 35,
        'B': 30,
        'C': 25,
        'D': 20,
        'E': 15,
        'F': 10,
    }
    return los_to_velocity[los] or 45


def parse_date_and_period(timestamp):
    ts = dt.datetime.fromtimestamp(timestamp)
    date, time, weeekday = ts.date(), ts.time(), ts.weekday()

    h, m, s = time.hour, time.minute, time.second

    hour = f"0{h}" if h < 10 else str(h)
    step = (m * 60 + s) // (PERIOD_LENGTH * 60)
    m = PERIOD_LENGTH * step
    minute = f"0{m}" if m < 10 else str(m)
    period = f"period_{hour}_{minute}"
    is_morning = 1 if h<=12 else 0
    return str(date), period, weeekday, is_morning


def reset():
    shutil.rmtree(TRAIN_DIR)

def get_period_from_timestamp(timestamp):
	timestamp = dt.datetime.fromtimestamp(timestamp)
	hour = timestamp.hour
	minute = timestamp.minute

	if (hour >= 0 and hour <= 5) or (hour >= 9 and hour <= 15) or (hour >= 19 and hour <= 23):
		return "period_{hour}".format(hour=hour)
	if (hour == 24):
		return 'period_0'
	if (minute >= 30):
		return "period_{hour}_30".format(hour=hour)
	return "period_{hour}".format(hour=hour)

def get_seg_weather_data(timestamp, weather_data):
    weather = ""
    temperature = ""
    try:
        time_data = weather_data[str(timestamp)]
        weather = time_data["weather"][0]["main"]
        temperature = time_data["main"]["temp"]
    except:
        pass
    return weather, temperature

In [43]:
seg_info_df = pd.read_csv("segments_with_district.csv", index_col=0)


In [100]:
temp_df = seg_info_df.loc[seg_info_df["_id"] == 21516].iloc[-1]
temp_df["district"]

'Quận 10'

In [83]:
checked_time_list = {
   "1": ["15:57:26", "18:57:26"],
   "2": ["6:28:00", "10:32:00"]
}

In [45]:
def check_times(need_to_checked_time):
    global checked_time_list
    checked_time = False
    for key, value in checked_time_list.items():
        if convert_str_to_time(value[0]) <= need_to_checked_time <= convert_str_to_time(value[1]):
            checked_time = True

    return checked_time

In [46]:
def convert_str_to_time(time_str):
    return dt.datetime.strptime(time_str, '%H:%M:%S').time()

In [103]:
def _get_df_from_json_file(json_file_path):
    output = []
    
    try:
        f = json_file_path.split('/')[-1]
        timestamp = f.split('.')[0]
        date, period, weekday, is_morning = parse_date_and_period(int(timestamp))
        split_period = period.split('_')
        str_time = f"{split_period[1]}:{split_period[2]}:00"
        convert_time = convert_str_to_time(str_time)
        if not check_times(convert_time):
            print(date, period, weekday, is_morning)
            return 0
        
        with open(json_file_path, 'r') as f:
            json_data = json.load(f)
            for k, v in json_data.items():
                isHot = True
                base_LOS = velocity_to_los(v['velocity'])
                temp_df = seg_info_df.loc[seg_info_df["_id"] == int(k)].iloc[0]
                output.append([period, k, date, 
                    weekday, v['velocity'], base_LOS, isHot, "few clouds", 27, temp_df["district"], temp_df["lat"], temp_df["lng"], is_morning])
            
    except Exception as error:
        print(error)
        return 0
    
    folder_path = os.path.join(TRAIN_DIR, date)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    header = ['period', 'segment_id',
                'date', 'weekday', 'tomtom_velocity', 'base_LOS', 'isHot', 'weather', 'temperature', 'district', 'lat', 'lng', 'is_morning']
    df_result = pd.DataFrame(output, columns=header)
    path_csv_file  = os.path.join(folder_path, f'{period}.csv')
    df_result.to_csv(path_csv_file)
        

In [104]:
list_file_json = os.listdir('tom-tom-data')
for i in list_file_json:
    json_path = os.path.join("tom-tom-data", i)
    _get_df_from_json_file(json_path)
    

2022-05-10 period_00_05 1 1
2022-03-21 period_22_10 0 0
2022-03-27 period_14_10 6 0
2022-05-07 period_01_55 5 1
2022-05-04 period_02_25 2 1
2022-04-16 period_13_30 5 0
2022-04-30 period_13_15 5 0
2022-03-26 period_02_45 5 1
2022-04-29 period_14_55 4 0
2022-04-05 period_22_50 1 0
2022-04-07 period_03_50 3 1
2022-04-23 period_19_15 5 0
2022-05-04 period_11_20 2 1
2022-04-22 period_00_50 4 1
2022-04-01 period_22_50 4 0
2022-03-29 period_19_10 1 0
2022-04-09 period_10_35 5 1
2022-04-11 period_20_35 0 0
2022-05-04 period_04_45 2 1
2022-04-19 period_19_50 1 0
2022-03-25 period_19_50 4 0
2022-03-28 period_06_25 0 1
2022-03-31 period_01_20 3 1
2022-04-03 period_02_40 6 1
2022-04-06 period_03_55 2 1
2022-04-22 period_12_25 4 1
2022-04-24 period_14_10 6 0
2022-04-04 period_15_35 0 0
2022-04-23 period_02_10 5 1
2022-03-25 period_11_20 4 1
2022-04-10 period_13_40 6 0
2022-04-01 period_00_10 4 1
2022-03-29 period_01_00 1 1
2022-04-06 period_03_20 2 1
2022-05-06 period_22_05 4 0
2022-04-17 period_21

2022-04-08 period_13_35 4 0
2022-04-17 period_19_50 6 0
2022-04-07 period_12_15 3 1
2022-03-26 period_22_50 5 0
2022-04-29 period_23_05 4 0
2022-03-24 period_19_05 3 0
2022-04-19 period_11_15 1 1
2022-04-11 period_13_35 0 0
2022-03-23 period_19_05 2 0
2022-04-11 period_20_20 0 0
2022-04-23 period_20_00 5 0
2022-04-11 period_21_10 0 0
2022-03-30 period_22_30 2 0
2022-05-03 period_23_25 1 0
2022-04-09 period_12_20 5 1
2022-03-25 period_19_20 4 0
2022-03-29 period_14_00 1 0
2022-04-03 period_03_25 6 1
2022-04-07 period_22_00 3 0
2022-03-26 period_23_20 5 0
2022-04-20 period_02_25 2 1
2022-04-04 period_06_05 0 1
2022-03-24 period_11_30 3 1
2022-03-29 period_15_00 1 0
2022-05-05 period_20_20 3 0
2022-05-07 period_22_30 5 0
2022-04-24 period_02_45 6 1
2022-04-27 period_02_20 2 1
2022-04-19 period_01_20 1 1
2022-03-29 period_12_05 1 1
2022-05-06 period_12_10 4 1
2022-04-10 period_05_35 6 1
2022-04-06 period_14_15 2 0
2022-04-23 period_19_20 5 0
2022-05-07 period_04_40 5 1
2022-04-04 period_21

2022-04-30 period_03_15 5 1
2022-03-30 period_05_10 2 1
2022-04-03 period_11_00 6 1
2022-04-10 period_04_00 6 1
2022-04-10 period_21_25 6 0
2022-04-19 period_01_45 1 1
2022-05-03 period_15_55 1 0
2022-05-02 period_05_25 0 1
2022-04-22 period_02_55 4 1
2022-03-23 period_20_10 2 0
2022-05-09 period_04_55 0 1
2022-03-27 period_02_40 6 1
2022-04-04 period_06_10 0 1
2022-04-27 period_13_35 2 0
2022-04-19 period_00_45 1 1
2022-04-09 period_04_15 5 1
2022-04-09 period_14_00 5 0
2022-03-24 period_03_25 3 1
2022-03-29 period_13_15 1 0
2022-04-22 period_02_25 4 1
2022-05-05 period_00_05 3 1
2022-05-02 period_05_45 0 1
2022-03-30 period_03_35 2 1
2022-04-29 period_22_45 4 0
2022-03-29 period_04_15 1 1
2022-04-03 period_22_45 6 0
2022-04-28 period_14_40 3 0
2022-04-24 period_15_35 6 0
2022-03-29 period_20_05 1 0
2022-04-17 period_14_45 6 0
2022-04-22 period_13_25 4 0
2022-05-05 period_19_25 3 0
2022-04-25 period_12_45 0 1
2022-04-02 period_13_10 5 0
2022-04-07 period_15_45 3 0
2022-04-01 period_20

2022-05-06 period_12_50 4 1
2022-05-01 period_05_40 6 1
2022-04-16 period_19_00 5 0
2022-04-10 period_03_30 6 1
2022-04-06 period_14_55 2 0
2022-04-07 period_23_35 3 0
2022-03-21 period_22_00 0 0
2022-05-01 period_04_00 6 1
2022-04-10 period_14_40 6 0
2022-04-07 period_04_20 3 1
2022-05-05 period_04_50 3 1
2022-04-26 period_05_45 1 1
2022-04-22 period_15_45 4 0
2022-04-09 period_13_25 5 0
2022-03-28 period_01_05 0 1
2022-04-24 period_11_10 6 1
2022-03-25 period_02_55 4 1
2022-03-23 period_05_55 2 1
2022-04-16 period_19_20 5 0
2022-05-08 period_01_25 6 1
2022-05-05 period_06_15 3 1
2022-05-01 period_13_35 6 0
2022-03-30 period_14_35 2 0
2022-05-08 period_19_45 6 0
2022-04-03 period_23_15 6 0
2022-05-06 period_04_00 4 1
2022-03-24 period_23_40 3 0
2022-03-24 period_04_45 3 1
2022-04-11 period_12_20 0 1
2022-03-24 period_14_25 3 0
2022-04-03 period_06_10 6 1
2022-03-20 period_19_10 6 0
2022-04-26 period_12_50 1 1
2022-04-04 period_21_50 0 0
2022-04-07 period_04_35 3 1
2022-04-17 period_03

2022-03-23 period_02_05 2 1
2022-04-20 period_03_00 2 1
2022-03-31 period_00_45 3 1
2022-03-24 period_20_20 3 0
2022-05-01 period_00_10 6 1
2022-03-28 period_04_00 0 1
2022-03-24 period_12_45 3 1
2022-05-07 period_04_30 5 1
2022-04-03 period_06_05 6 1
2022-05-02 period_15_55 0 0
2022-04-03 period_00_40 6 1
2022-04-17 period_02_55 6 1
2022-04-17 period_22_30 6 0
2022-03-25 period_05_20 4 1
2022-03-26 period_03_50 5 1
2022-04-02 period_04_10 5 1
2022-03-23 period_10_35 2 1
2022-03-30 period_12_30 2 1
2022-04-03 period_06_25 6 1
2022-04-29 period_00_40 4 1
2022-04-28 period_05_35 3 1
2022-04-11 period_15_20 0 0
2022-04-05 period_13_55 1 0
2022-05-01 period_11_55 6 1
2022-04-17 period_22_55 6 0
2022-04-20 period_13_15 2 0
2022-05-07 period_12_20 5 1
2022-04-26 period_11_15 1 1
2022-04-19 period_02_45 1 1
2022-04-28 period_14_30 3 0
2022-03-27 period_05_40 6 1
2022-03-22 period_21_50 1 0
2022-04-17 period_15_00 6 0
2022-04-10 period_04_50 6 1
2022-04-26 period_13_55 1 0
2022-03-27 period_15

...

2022-04-20 period_00_40 2 1
2022-04-25 period_23_30 0 0
2022-03-27 period_04_05 6 1
2022-04-06 period_01_20 2 1
2022-04-22 period_04_45 4 1
2022-04-10 period_00_00 6 1
2022-03-24 period_12_05 3 1
2022-04-02 period_19_35 5 0
2022-05-10 period_05_35 1 1
2022-04-05 period_14_55 1 0
2022-03-26 period_20_20 5 0
2022-05-08 period_04_10 6 1
2022-05-02 period_05_35 0 1
2022-04-23 period_01_00 5 1
2022-03-22 period_00_30 1 1
2022-03-27 period_21_55 6 0
2022-04-26 period_14_40 1 0
2022-05-03 period_04_55 1 1
2022-04-26 period_04_45 1 1
2022-04-01 period_00_05 4 1
2022-04-30 period_03_50 5 1
2022-04-17 period_20_20 6 0
2022-04-27 period_05_20 2 1
2022-04-26 period_14_20 1 0
2022-03-31 period_05_45 3 1
2022-03-23 period_22_50 2 0
2022-03-25 period_04_00 4 1
2022-05-07 period_12_15 5 1
2022-05-05 period_12_05 3 1
2022-04-11 period_03_15 0 1
2022-04-20 period_12_35 2 1
2022-04-03 period_11_50 6 1
2022-04-04 period_10_45 0 1
2022-04-23 period_13_40 5 0
2022-04-22 period_05_10 4 1
2022-05-03 period_06

2022-03-22 period_01_55 1 1
2022-04-23 period_23_30 5 0
2022-04-20 period_03_15 2 1
2022-04-27 period_21_10 2 0
2022-03-30 period_00_05 2 1
2022-04-28 period_19_45 3 0
2022-03-22 period_13_25 1 0
2022-04-19 period_04_25 1 1
2022-03-22 period_01_35 1 1
2022-04-24 period_20_35 6 0
2022-04-07 period_22_45 3 0
2022-04-26 period_04_05 1 1
2022-04-02 period_13_15 5 0
2022-05-09 period_01_55 0 1
2022-03-25 period_22_45 4 0
2022-04-07 period_04_45 3 1
2022-03-30 period_23_45 2 0
2022-03-23 period_19_10 2 0
2022-04-04 period_11_10 0 1
2022-05-03 period_15_15 1 0
2022-04-03 period_14_35 6 0
2022-04-03 period_01_00 6 1
2022-03-28 period_19_20 0 0
2022-05-06 period_03_05 4 1
2022-04-11 period_11_25 0 1
2022-05-07 period_04_35 5 1
2022-05-09 period_22_25 0 0
2022-04-24 period_21_00 6 0
2022-03-26 period_21_35 5 0
2022-04-26 period_00_25 1 1
2022-04-01 period_14_00 4 0
2022-04-05 period_20_30 1 0
2022-05-01 period_06_00 6 1
2022-04-01 period_03_30 4 1
2022-04-17 period_00_40 6 1
2022-04-10 period_02

2022-03-24 period_23_25 3 0
2022-04-06 period_12_00 2 1
2022-05-02 period_05_55 0 1
2022-04-25 period_12_35 0 1
2022-04-18 period_14_20 0 0
2022-04-16 period_23_30 5 0
2022-04-23 period_04_35 5 1
2022-05-04 period_06_00 2 1
2022-04-03 period_01_45 6 1
2022-04-04 period_00_50 0 1
2022-05-06 period_22_15 4 0
2022-03-28 period_04_25 0 1
2022-05-08 period_19_20 6 0
2022-04-02 period_03_20 5 1
2022-03-24 period_06_20 3 1
2022-04-21 period_22_20 3 0
2022-03-24 period_13_15 3 0
2022-04-24 period_03_45 6 1
2022-04-16 period_19_50 5 0
2022-05-08 period_15_40 6 0
2022-04-21 period_05_30 3 1
2022-04-07 period_01_45 3 1
2022-05-09 period_05_40 0 1
2022-04-19 period_02_25 1 1
2022-05-06 period_01_40 4 1
2022-04-20 period_05_35 2 1
2022-04-05 period_05_40 1 1
2022-05-01 period_04_25 6 1
2022-05-08 period_23_55 6 0
2022-04-19 period_23_15 1 0
2022-04-18 period_12_05 0 1
2022-03-26 period_22_35 5 0
2022-04-23 period_12_25 5 1
2022-03-25 period_22_55 4 0
2022-04-08 period_21_55 4 0
2022-04-29 period_15

2022-04-09 period_15_45 5 0
2022-04-17 period_15_05 6 0
2022-03-31 period_23_00 3 0
2022-04-26 period_04_00 1 1
2022-04-17 period_21_10 6 0
2022-04-27 period_11_55 2 1
2022-03-29 period_03_45 1 1
2022-04-23 period_01_20 5 1
2022-05-06 period_23_45 4 0
2022-05-09 period_02_00 0 1
2022-04-30 period_04_15 5 1
2022-05-09 period_20_40 0 0
2022-04-25 period_21_35 0 0
2022-03-30 period_13_00 2 0
2022-04-05 period_12_15 1 1
2022-03-29 period_23_00 1 0
2022-04-05 period_14_10 1 0
2022-05-08 period_15_25 6 0
2022-04-05 period_10_35 1 1
2022-04-08 period_14_55 4 0
2022-04-04 period_13_00 0 0
2022-04-08 period_23_50 4 0
2022-04-16 period_19_25 5 0
2022-04-20 period_04_15 2 1
2022-04-20 period_00_05 2 1
2022-03-31 period_14_00 3 0
2022-03-25 period_00_05 4 1
2022-05-08 period_20_30 6 0
2022-05-04 period_20_10 2 0
2022-04-19 period_12_55 1 1
2022-03-27 period_19_15 6 0
2022-04-04 period_12_45 0 1
2022-05-01 period_23_55 6 0
2022-04-10 period_12_40 6 1
2022-04-08 period_21_40 4 0
2022-03-22 period_14

2022-03-28 period_05_10 0 1
2022-04-30 period_01_25 5 1
2022-04-02 period_12_30 5 1
2022-04-12 period_15_35 1 0
2022-04-11 period_05_00 0 1
2022-05-09 period_00_05 0 1
2022-04-22 period_03_00 4 1
2022-04-29 period_13_05 4 0
2022-05-01 period_21_55 6 0
2022-04-29 period_13_30 4 0
2022-05-04 period_03_40 2 1
2022-03-25 period_00_35 4 1
2022-03-25 period_15_20 4 0
2022-04-09 period_15_35 5 0
2022-04-08 period_00_25 4 1
2022-04-27 period_00_25 2 1
2022-05-07 period_23_10 5 0
2022-04-17 period_06_10 6 1
2022-04-30 period_19_10 5 0
2022-04-18 period_02_40 0 1
2022-04-11 period_12_45 0 1
2022-04-02 period_21_25 5 0
2022-03-26 period_11_30 5 1
2022-04-06 period_03_10 2 1
2022-04-04 period_05_20 0 1
2022-04-30 period_05_00 5 1
2022-05-05 period_21_40 3 0
2022-05-03 period_00_15 1 1
2022-04-19 period_20_30 1 0
2022-04-21 period_23_55 3 0
2022-03-21 period_23_50 0 0
2022-05-04 period_13_30 2 0
2022-04-05 period_13_15 1 0
2022-04-19 period_22_30 1 0
2022-03-27 period_01_55 6 1
2022-04-24 period_03

In [3]:
def get_duration(x, df_data):
    try:
        temp_data = df_data.loc[(df_data.segment_id == x.segment_id)&(df_data.is_morning == x.is_morning)].sort_values(by=['period'], ascending = False).iloc[-1]
        if len(temp_data) > 0:
            if temp_data.base_LOS == x.base_LOS:
                result_duration = int(temp_data.duration) + 1
            else:
                result_duration = 1
        else:
    #         print(temp_data.segment_id)
            result_duration = 1
    except:
        result_duration = 1
    return result_duration
def get_duration_with_velocity(x, df_data):
    try:
        temp_data = df_data.loc[(df_data.segment_id == x.segment_id)&(df_data.is_morning == x.is_morning)&(df_data.date == x.date)].sort_values(by=['period'], ascending = False).iloc[-1]
        if len(temp_data) > 0:
            if temp_data.tomtom_velocity == x.tomtom_velocity:
                result_duration = int(temp_data.duration) + 1
            else:
                result_duration = 1
        else:
    #         print(temp_data.segment_id)
            result_duration = 1
    except:
        result_duration = 1
    return result_duration

In [11]:
ready_file_list = os.listdir('result_df')
print(type(ready_file_list))
a = '2022-04-11'
if f'{a}.csv' in list(ready_file_list):
    print("--")

<class 'list'>
--


In [12]:
list_folder = os.listdir(TRAIN_DIR)
for folder_item in list_folder:
    if f"{folder_item}.csv" in ready_file_list:
        print(folder_item)
        continue
    print("-----", folder_item)
    temp_list_file = os.listdir(os.path.join(TRAIN_DIR, folder_item))
    temp_list_file = sorted(temp_list_file, reverse=True)
    result_df = pd.read_csv(os.path.join(TRAIN_DIR, folder_item, temp_list_file[0]))
    for index, csv_item in enumerate(temp_list_file):
        temp_df = pd.read_csv(os.path.join(TRAIN_DIR, folder_item, csv_item))
        if index == 0:
            result_df['duration'] = [1]*temp_df.shape[0]
        else:
            duration_list = temp_df.apply(get_duration_with_velocity, axis=1, args=(result_df,))
            temp_df['duration'] = duration_list
            result_df = pd.concat([result_df, temp_df])
    TRAIN_DIR2 = "result_df"
    folder_path = os.path.join(TRAIN_DIR2)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    print(folder_item)
    result_df.to_csv(os.path.join(folder_path, f"{folder_item}.csv"))
    
        

2022-04-11
2022-04-16
2022-04-29
2022-04-20
2022-04-27
2022-03-30
2022-04-18
2022-03-31
2022-04-26
2022-04-19
2022-04-21
2022-04-17
2022-04-28
2022-04-10
2022-05-10
2022-05-05
2022-05-02
2022-05-03
2022-05-04
2022-03-25
2022-03-22
2022-04-03
2022-04-04
2022-04-05
2022-04-02
2022-03-23
2022-03-24
2022-04-12
2022-04-24
2022-04-23
2022-04-22
2022-04-25
2022-05-01
2022-05-06
2022-05-08
2022-05-09
2022-05-07
2022-04-09
2022-03-21
2022-03-26
2022-04-07
2022-03-28
----- 2022-03-29
2022-03-29
----- 2022-04-01
2022-04-01
----- 2022-04-06
2022-04-06
----- 2022-04-30
2022-04-30
----- 2022-03-27
2022-03-27
----- 2022-04-08
2022-04-08
----- 2022-03-20
2022-03-20


In [6]:
list_date = ["2023-04-19", "2023-04-20", "2023-04-21", "2023-04-22", "2023-04-23", "2023-04-24", "2023-04-25", "2023-04-26", "2023-04-27", "2023-04-28", "2023-04-29", "2023-04-30", "2023-05-01", "2023-05-02", "2023-05-03", "2023-05-04", "2023-05-05", "2023-05-06", "2023-05-07", "2023-05-08", "2023-05-09", "2023-05-10", "2023-05-11", "2023-05-12", "2023-05-13", "2023-05-14", "2023-05-15", "2023-05-16"]
for date_item in list_date:
    folder_path = 'tomtom-voh/' + date_item
    list_file = define.bucket.objects.filter(Prefix=folder_path)
    list_path_files = []
    print(date_item)
    for obj in list_file:
        f = obj.key.split('/')[-1]
        timestamp = f.split('.')[0]
        date, period, weekday, _ = parse_date_and_period(int(timestamp))
        list_path_files.append(
            {
                "key":period,
                "file_path": obj.key
            }
        )
    list_path_files = sorted(list_path_files, key=lambda k: k['key']  , reverse=True)
    df_result = get_result_df(list_path_files)
    df_file_name = date_item + ".csv"
    df_result.to_csv(os.path.join(TRAIN_DIR, df_file_name))
    s3_key = os.path.join("Result_with_velocity", df_file_name)
    define.upload_file_to_s3(os.path.join(TRAIN_DIR, df_file_name),s3_key)

2023-04-19
2023-04-20
2023-04-21
2023-04-22
2023-04-23
2023-04-24
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-04-29
2023-04-30
2023-05-01
2023-05-02
2023-05-03
2023-05-04
2023-05-05
2023-05-06
2023-05-07
2023-05-08
2023-05-09
2023-05-10
2023-05-11
2023-05-12
2023-05-13
2023-05-14
2023-05-15
2023-05-16


In [184]:
list_path_files = sorted(list_path_files, key=lambda k: k['key']  , reverse=True)

In [194]:
df_result = get_result_df(list_path_files)

period_19_55
period_19_55
period_19_50
period_19_45
period_19_40
period_19_30
period_19_25
period_19_20
period_19_15
period_19_10
period_19_05
period_19_00
period_18_55
period_18_50
period_18_45
period_18_40
period_18_35
period_18_30
period_18_25
period_18_20
period_18_15
period_18_10
period_18_05
period_18_00
period_17_55
period_17_50
period_17_45
period_17_40
period_17_35
period_17_30
period_17_25
period_17_20
period_17_15
period_17_10
period_17_05
period_17_00
period_16_55
period_16_50
period_16_45
period_16_40
period_16_30
period_16_25
period_16_20
period_16_15
period_16_10
period_16_05
period_16_00
period_15_55
period_10_25
period_10_20
period_10_15
period_10_10
period_10_05
period_10_00
period_09_55
period_09_50
period_09_45
period_09_40
period_09_35
period_09_30
period_09_25
period_09_20
period_09_15
period_09_10
period_09_05
period_09_00
period_08_55
period_08_50
period_08_45
period_08_40
period_08_35
period_08_30
period_08_25
period_08_20
period_08_15
period_08_10
period_08_05

In [203]:
date_item = "2023-04-23"
df_file_name = date_item + ".csv"
df_result.to_csv(os.path.join(TRAIN_DIR, df_file_name))

In [204]:
s3_key = os.path.join("Result", df_file_name)
define.upload_file_to_s3(os.path.join(TRAIN_DIR, df_file_name),s3_key)